### 필요한 라이브러리들

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import pandas as pd

### 1) 제품군 선택해서 get에 해당 제품군에 대한 링크 할당
- 선택한 제품군을 Bestselling 순으로 정렬한 페이지 get
- TV : 'https://www.amazon.in/s?bbn=1389396031&rh=n%3A1389396031%2Cp_89%3ALG&dc&qid=1724652002&rnid=3837712031&ref=lp_1389396031_nr_p_89_0'
- 세탁기 : 'https://www.amazon.in/s?bbn=1380369031&rh=n%3A1380369031%2Cp_89%3ALG&dc&qid=1724689251&rnid=3837712031&ref=lp_1380369031_nr_p_89_2'
- 에어컨 : 'https://www.amazon.in/s?bbn=3474656031&rh=n%3A3474656031%2Cp_89%3ALG&dc&qid=1724689377&rnid=3837712031&ref=lp_3474656031_nr_p_89_2'
- 냉장고 : 'https://www.amazon.in/s?bbn=1380365031&rh=n%3A1380365031%2Cp_89%3ALG&dc&qid=1724689330&rnid=3837712031&ref=lp_1380365031_nr_p_89_2'

In [2]:
driver=webdriver.Chrome()
driver.get('https://www.amazon.in/s?bbn=1389396031&rh=n%3A1389396031%2Cp_89%3ALG&dc&qid=1724652002&rnid=3837712031&ref=lp_1389396031_nr_p_89_0')
driver.maximize_window()

### 2) 각각의 제품 모델에 대한 상세 페이지 접근
- url 수집 및 저장
- 최대 24개의 제품 모델

In [3]:
soup = bs(driver.page_source, 'lxml')
# url 접근 위한 select
links = soup.select('h2 > a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
# 모델명 데이터 수집 위한 select
names = soup.select('span.a-size-base-plus.a-color-base.a-text-normal')

link_list = []
for link in links :
    link_list.append("https://www.amazon.in"+link['href'])

name_list = []
for name in names :
    name_list.append(name.text)

# name_list # 모델명 확인용 
# link_list # url 확인용

+ 리뷰별 처리된 데이터 수집

In [4]:
import pandas as pd
import re

#전처리 함수
def appendList(spect_list):

    keyword_list.append(re.findall(r'"(.*?)"', spect_list[0]))

    m=int(spect_list[0][:spect_list[0].find(' ')].replace(',', ''))
    mention_list.append(m)

    p=int(spect_list[1][:spect_list[1].find(' ')].replace(',', ''))
    positive_list.append(p)

    n=int(spect_list[2][:spect_list[2].find(' ')].replace(',', ''))
    negative_list.append(n)

    nega_percent_list.append(round(n/m,2))
    
    comment_list.append(spect_list[3])
    
    example_list=[]
    
    for line in spect_list[4:]:
        example_list.append(re.findall(r'"(.*?)"', line)[0])
        
    examples_list.append(example_list)    

In [5]:
product_name_list=[]
keyword_list=[]
mention_list=[]
positive_list=[]
negative_list=[]
nega_percent_list=[]
comment_list=[]
examples_list=[]

In [6]:
for link in link_list:
    driver=webdriver.Chrome()
    #드라이버에 리뷰페이지 링크 넣기
    driver.get(link)
    driver.maximize_window()
    text_list=[]
    time.sleep(1)

    #제품명
    id = driver.find_elements(By.CLASS_NAME,"a-size-base.prodDetAttrValue")
    # name = id[1].text
    name = id[2].text #TV
    
    for i in range(10): #가능한 만큼 for문 돌아가면서 클릭함
        try:
            #키워드 버튼 클릭
            aspect = driver.find_element(By.ID,f"aspect-button-0-{i}")
            aspect.click()
            time.sleep(1)
            
            #클릭 상태에서 본문 뽑기
            spect = driver.find_element(By.ID,f"aspect-bottom-sheet-0-{i}")
            text_list.append(spect.text)
            
        except:
            #전처리
            for txt in text_list: 
                if txt !='':
                    spect_list=txt.split('\n')
                    appendList(spect_list)
                    product_name_list.append(name) #길이 지정해주기 위해 제품명은 따로 append
            break

- 최종 결과 데이터프레임

In [7]:
data = {
    'product_name': product_name_list,
    'keyword': keyword_list,
    'mention': mention_list,
    'positive': positive_list,
    'negative': negative_list,
    'neg_percent': nega_percent_list,
    'comment': comment_list,
    'examples': examples_list
}

df = pd.DataFrame(data)
df

,product_name,keyword,mention,positive,negative,neg_percent,comment,examples
0,32LM563BPTC,[Quality],651,591,60,0.09,Customers are satisfied with the quality of th...,[...from the text of customer reviews and it i...
1,32LM563BPTC,[Picture quality],432,338,94,0.22,Customers like the picture quality of the tele...,[...AI-generated from the text of customer rev...
2,32LM563BPTC,[Value],300,234,66,0.22,Customers like the value of the television. Th...,"[Customers like the picture quality, quality a..."
3,32LM563BPTC,[Sound quality],309,207,102,0.33,Customers are mixed about the sound quality of...,"[...That said, opinions are mixed on performan..."
4,32LM563BPTC,[Service],161,99,62,0.39,Customers are mixed about the service. Some me...,[...AI-generated from the text of customer rev...
...,...,...,...,...,...,...,...,...
86,OLED48A3PSA,[Quality],3,3,0,0.00,Customers are satisfied with the quality of th...,[...Technically sound product. No complaints!....
87,OLED48A3PSA,[Missing pieces],4,0,4,1.00,Customers have issues with the missing stand a...,"[...Only issue faced during installation was, ..."
88,55LX1QPSA,[Picture quality],5,5,0,0.00,Customers are satisfied with the picture quali...,[...the TV now stands proudly in my living roo...
89,55LX1QPSA,[Quality],3,3,0,0.00,Customers are satisfied with the quality of th...,[...Technically sound product. No complaints!....


In [8]:
df.to_csv('Amazon_LG_TV_insight_review.csv')
# df.to_csv('Amazon_LG_Washing_Machines_insight_review.csv')
# df.to_csv('Amazon_LG_Air_Conditioner_insight_review.csv')
# df.to_csv('Amazon_LG_Refrigerator_insight_review.csv')

# 제품명과 링크 저장
target_df = pd.DataFrame({
    'Product Name': name_list,
    'Link': link_list
})

target_df.to_csv('Amazon_LG_TV_insight_target.csv')
# target_df.to_csv('Amazon_LG_Washing_Machines_insight_target.csv')
# target_df.to_csv('Amazon_LG_Air_Conditioner_insight_target.csv')
# target_df.to_csv('Amazon_LG_Refrigerator_insight_target.csv')